In [1]:
import numpy as np
import pandas as pd

# %pip install -U pymoo

### Parámetros de entrada
Información que obtiene el modelo del sistema de información

In [2]:
# Demanda
DEMANDA = 60
# Costo por unidad de tiempo
CTIEMPO = 100
# Tiempo máximo definido
T_MAX = 360

# Parámetros de entrada
params_df = pd.read_excel('./data/' + 'nuevo_info_acopios.xlsx')

# Matriz de costos de transporte
trans_cost_df = pd.read_excel('./data/' + 'costoTransporteCAi.xlsx', index_col=0)

# Matriz de tiempo de transporte
trans_time_df = pd.read_excel('./data/' + 'tiempoTransporteCAi.xlsx', index_col=0)

### Variables
Definición de variables para el modelo de optimización

In [3]:
N = params_df.shape[0]
CAPACIDADES = np.empty(N*2, dtype=float)

for cap_i in range(0, N*2, 2):
    CAPACIDADES[cap_i] = params_df['Stock'].iloc[cap_i//2]
    CAPACIDADES[cap_i+1] = params_df['Ppotencial'].iloc[cap_i//2]
    
CAPACIDADES = np.append(CAPACIDADES, N-1)
CAPACIDADES

array([10.   ,  0.5  , 12.   ,  0.625,  4.   , 14.   ,  8.   , 13.   ,
       18.   , 15.   ,  6.   , 14.   , 12.   , 13.   ,  2.   , 12.5  ,
       30.   , 15.   , 15.   , 18.   ,  9.   ])

# Función objetivo
$$
\begin{align*}
    \sum_{i=1 \quad i\neq p}^{N} &\big[ \, kCA_i \times Precio(CA_i) + kCA_i \times cTransp(CA_i) + Tiempo(CA_i) \times cTiempo \, \big] \, + \\
                                 &\big[ \, kCA_p \times Precio(CA_p) + Demanda \times cTransp(CA_p) + Tiempo(CA_p) \times cTiempo \, \big]
\end{align*}
$$

In [26]:
def f(x):
    delta = 0
    idx_principal = int(x[N*2])
    
    for i in range(0, N*2, 2):
        idx_acopio = i//2

        if i == idx_principal or (x[i] == 0 and x[i+1] == 0):
            continue
            
        delta += get_delta(x, idx_acopio, idx_principal)

    delta += get_delta(x, idx_principal)
    
    return delta

def get_delta(x, idx_acopio, idx_principal=-1):
    kca = x[idx_acopio] + x[idx_acopio + 1]
    precio = params_df['Precio'].iloc[idx_acopio]
    talistam = 0
    
    if x[idx_acopio + 1]:
        talistam = params_df['TiempoAlistam'].iloc[idx_acopio]

    ' Si es el centro de acopio principal (idx_principal=-1) '
    if idx_principal < 0:
        ctransp = params_df['Ctransp'].iloc[idx_acopio]
        ttransp = params_df['TiempoTransp'].iloc[idx_acopio]
        tiempo = talistam + ttransp
        
        return (kca * precio) + (DEMANDA * ctransp) + (tiempo * CTIEMPO)
    else:
        ctransp = trans_cost_df.iloc[idx_acopio, idx_principal]
        ttransp = trans_time_df.iloc[idx_acopio, idx_principal]
        tiempo = talistam + ttransp
        
        return (kca * precio) + (kca * ctransp) + (tiempo * CTIEMPO)
  
test_arr = np.array([0, 0, 0, 0, 0, 0, 4, 11, 0, 0, 0, 0, 0, 0, 0, 0, 30, 15, 0, 0, 8])  
print(f(test_arr))

875580


Sujeta a las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{N} kCA_i &= Demanda \\
    kCA_i &\leq Stock(CA_i) + Ppotencial(CA_i) &\therefore \, i=0,\cdots ,N \\
    TiempoAlistam(CA_i) &\leq TiempoMaxDefinido &\therefore \, i=0,\cdots ,N \\
    Tiempo(CA_i) &= TiempoAlistam(CA_i) + TiempoTransp(CA_i) \\
\end{align*}
$$

In [5]:
# Función para alterar el vector para que cumpla con la restricción de igualdad
def balance(individual, delta, diff):
    delta = np.squeeze(delta)
    
    # Reducir para igualar a la demanda
    if diff:
        acopios = list(np.nonzero(individual)[0])

        while delta > 0 and len(acopios) > 0:
            idx = np.random.choice(acopios)
            acopios.remove(idx)

            if delta <= individual[idx]:
                individual[idx] -= delta
                delta = 0
            else:
                delta -= individual[idx]
                individual[idx] = 0
    # Aumentar para igualar a la demanda
    else:
        acopios = list(np.where(individual == 0)[0])

        while delta > 0 and len(acopios) > 0:
            idx = np.random.choice(acopios)
            acopios.remove(idx)

            if delta <= CAPACIDADES[idx]:
                individual[idx] = delta
                delta = 0
            else:
                individual[idx] = CAPACIDADES[idx]
                delta -= CAPACIDADES[idx]

## pymoo

Se evalúa el modelo tomando cada centro de acopio como el principal, y la ejecución da resultados para cada uno, luego de todas las soluciones se toma la mejor solución.

In [6]:
from pymoo.core.problem import ElementwiseProblem

xl = np.zeros(CAPACIDADES.shape[0])
xu = CAPACIDADES

' Definición del modelo de optimización '
class Queso(ElementwiseProblem):
    ca_principal = None
    
    def __init__(self, principal=0):
        self.ca_principal = principal
        
        super().__init__(
            n_var=len(xl),
            n_obj=1,
            n_eq_constr=1,
            # n_ieq_constr=1,
            xl=xl,
            xu=xu
        )
        
        
    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = f(x)
        out['H'] = DEMANDA - np.sum(x)

model = Queso()

## Generación 
De nuevas posibles soluciones

In [7]:
from pymoo.core.sampling import Sampling

class TopOrZeroSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        gen_matrix = np.zeros((n_samples, problem.n_var), dtype=float)
        n_vars = problem.n_var

        for i in range(n_samples):
            indices = np.arange(n_vars-1)
            np.random.shuffle(indices)

            while np.sum(gen_matrix[i]) < DEMANDA and indices.size > 0:
                idx = indices[0]
                gen_matrix[i, idx] += CAPACIDADES[idx]
                indices = np.delete(indices, 0)

                if np.sum(gen_matrix[i]) > DEMANDA:
                    gen_matrix[i, idx] = gen_matrix[i, idx] - (np.sum(gen_matrix[i]) - DEMANDA)
                    break

            gen_matrix[i, N*2] = np.random.randint(CAPACIDADES[N * 2] + 1)

        return gen_matrix

## Cruce
1. Para cada individuo en una muestra, se selecciona al azar otro individuo 
2. Se usa un índice (en medio) para separar el cruce de genes
3. Se crea el nuevo individuo 

In [8]:
from pymoo.core.crossover import Crossover

class SinglePointCross(Crossover):
    def __init__(self):
        # Parents, Offspring
        super().__init__(2, 1)

    def _do(self, problem, X, **kwargs):
        # n_parents, n_matings, n_var
        _, n_matings, n_var = X.shape

        T = np.zeros((1, n_matings, n_var))
        Y = np.full_like(T, None, dtype=float)

        for idx in range(n_matings):

            # Primera mitad
            p1 = X[0, idx, : n_var//2]
            # Segunda mitad
            p2 = X[1, idx, n_var//2 : n_var - 1]

            offspring = np.concatenate((p1, p2))

            if np.sum(offspring) > DEMANDA:
                delta = np.sum(offspring) - DEMANDA
                balance(offspring, delta, True)
            else:
                delta = DEMANDA - np.sum(offspring)
                balance(offspring, delta, False)

            for i in range(offspring.shape[0]):
                Y[0, idx, i] = offspring[i]

            # Centro de acopio principal (última posición)
            main = np.random.choice([X[0, idx, n_var-1], X[1, idx, n_var-1]])
            Y[0, idx, n_var-1] = main

        return Y

## Mutación
1. Tomar un índice (cacopio) aleatorio
2. **Si es cero:**
3. Cambiar a la cantidad máxima de la capacidad del centro de acopio
4. La cantidad que se asignó, restarla a lo largo de los demás centros de acopio para mantener la restricción de igualdad
5. **Sino:**
6. Cambiar a cero
7. La cantidad que se eliminó, sumarla en los demás centros de acopio para mantener la restricción de igualdad

In [9]:
from pymoo.core.mutation import Mutation

class ReassignMutation(Mutation):
    def __init__(self):
        super().__init__()

    def _do(self, problem, X, **kwargs):
        for i in range(len(X)):
            r = np.random.random()

            # Con una probabilidad de 40%, reasignar la cantidad de un centro de acopio
            if r < 0.4:
                individual = X[i, : problem.n_var-1]
                idx_mut = np.random.randint(individual.shape)
                # False = Menor
                # True = Mayor
                diff = False

                if individual[idx_mut] == 0:
                    delta = CAPACIDADES[idx_mut]
                    individual[idx_mut] = CAPACIDADES[idx_mut]
                    diff = True

                else:
                    delta = individual[idx_mut]
                    individual[idx_mut] = 0

                balance(individual, delta, diff)

                for j in range(individual.shape[0]):
                    X[i, j] = individual[j]

            # Con una probabilidad de 20%, reasignar el centro de acopio principal
            elif r < 0.6:
                X[i, problem.n_var - 1] = np.random.randint(CAPACIDADES[N * 2] + 1)

        return X

## Aplicación

In [20]:
# Función para mostrar la cantidad asignada y la total por centro de acopio
def print_acopios(X):
    for i in range(0, N*2, 2):
        idx_ca = i // 2
        print(f'Centro de acopio {params_df['Id_CA'].iloc[idx_ca]}:')
        print(f'Stock: {params_df['Stock'].iloc[idx_ca]}, \t\t\tAsignado: {X[i]}')
        print(f'Potencial: {params_df['Ppotencial'].iloc[idx_ca]}, \tAsignado: {X[i+1]}\n')

    print(f'Centro de acopio principal: {X[N*2]}')

In [19]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize

algorithm = GA(
    pop_size=N*100,
    sampling=TopOrZeroSampling(),
    crossover=SinglePointCross(),
    mutation=ReassignMutation(),
    eliminate_duplicates=True
)

res = minimize(model,
               algorithm,
               seed=1,
               verbose=False)

print_acopios(res.X)
print(f'Precio: {res.F}')

Centro de acopio CA1:
Stock: 10, 			Asignado: 0.0
Potencial: 0.5, 	Asignado: 0.0

Centro de acopio CA2:
Stock: 12, 			Asignado: 0.0
Potencial: 0.625, 	Asignado: 0.0

Centro de acopio CA3:
Stock: 4, 			Asignado: 4.0
Potencial: 14.0, 	Asignado: 11.0

Centro de acopio CA4:
Stock: 8, 			Asignado: 0.0
Potencial: 13.0, 	Asignado: 0.0

Centro de acopio CA5:
Stock: 18, 			Asignado: 0.0
Potencial: 15.0, 	Asignado: 0.0

Centro de acopio CA6:
Stock: 6, 			Asignado: 0.0
Potencial: 14.0, 	Asignado: 0.0

Centro de acopio CA7:
Stock: 12, 			Asignado: 0.0
Potencial: 13.0, 	Asignado: 0.0

Centro de acopio CA8:
Stock: 2, 			Asignado: 0.0
Potencial: 12.5, 	Asignado: 0.0

Centro de acopio CA9:
Stock: 30, 			Asignado: 30.0
Potencial: 15.0, 	Asignado: 15.0

Centro de acopio CA10:
Stock: 15, 			Asignado: 0.0
Potencial: 18.0, 	Asignado: 0.0

Centro de acopio principal: 0.0
Precio: [1317260.]


## scipy